In [3]:
import cv2
import time
import math
import mediapipe as mp
import numpy as np

In [19]:
def find_distance(a, b, w, h):
    a = np.array(a)
    b = np.array(b)
    a[0] = a[0] * w
    a[1] = a[1] * h
    b[0] = b[0] * w
    b[1] = b[1] * h
    dist = math.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)
    return dist

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [24]:
def get_angles(a, b, w, h):
    a = np.array(a)
    b = np.array(b)
    a[0] = a[0] * w
    a[1] = a[1] * h
    b[0] = b[0] * w
    b[1] = b[1] * h
    theta = math.acos((b[1] - a[1]) * (-a[1]) / (math.sqrt((b[0] - b[1]) ** 2 + (b[1] - a[1]) ** 2) * a[1]))
    degree = int(180/math.pi) * theta
    return degree

In [5]:
def send_warning(x):
    print("Warning: ", x)

In [20]:
good_frames = 0
bad_frames = 0

font = cv2.FONT_HERSHEY_SIMPLEX

blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)
black = (0, 0, 0)

In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [12]:
def extract_top_landmarks(landmarks, direction):
    if direction == "left":
        ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    if direction == "right":
        ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    return ear, shoulder, hip

In [40]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        fps = cap.get(cv2.CAP_PROP_FPS)
        h, w = frame.shape[:2]
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            l_ear, l_shoulder, l_hip = extract_top_landmarks(landmarks, "left")
            r_ear, r_shoulder, r_hip = extract_top_landmarks(landmarks, "right")
            offset = find_distance(l_shoulder, r_shoulder, w, h)
            if offset < 100:
                cv2.putText(frame, str(int(offset)) + " Aligned", (w-150, 30), font, 0.9, green, 2)
                neck_angle = get_angles(r_shoulder, r_ear, w, h)
                torso_angle = get_angles(r_hip, r_shoulder, w, h)
                angle_text_string = 'Neck : ' + str(int(neck_angle)) + '  Torso : ' + str(int(torso_angle))
                if neck_angle < 40 and torso_angle < 10:
                    bad_frames = 0
                    good_frames += 1
                    cv2.putText(frame, angle_text_string, (10, 30), font, 0.9, light_green, 2)
                    cv2.putText(frame, str(int(neck_angle)), (r_shoulder.x + 10, r_shoulder.y), font, 0.9, light_green, 2)
                    cv2.putText(frame, str(int(torso_angle)), (r_hip.x + 10, r_hip.y), font, 0.9, light_green, 2)
                else:
                    good_frames = 0
                    bad_frames += 1
                    cv2.putText(frame, angle_text_string, (10, 30), font, 0.9, red, 2)
                    cv2.putText(frame, str(int(neck_angle)), (r_shoulder.x + 10, r_shoulder.y), font, 0.9, red, 2)
                    cv2.putText(frame, str(int(torso_angle)), (r_hip.x + 10, r_hip.y), font, 0.9, red, 2)
                
                print("FPS: ", fps)

                good_time = (1/fps) * good_frames
                bad_time = (1/fps) * bad_frames
                print("Good Time: " + good_time)
                print("Bad Time: " + bad_time)

                if good_time > 0:
                    time_string_good = 'Good Posture Time : ' + str(round(good_time, 1)) + 's'
                    cv2.putText(frame, time_string_good, (10, h - 20), font, 0.9, green, 2)
                else:
                    time_string_bad = 'Bad Posture Time : ' + str(round(bad_time, 1)) + 's'
                    cv2.putText(frame, time_string_bad, (10, h - 20), font, 0.9, red, 2)
                if bad_time > 180:
                    send_warning("Bad Posture for 3 minutes")
            else:
                cv2.putText(frame, str(int(offset)) + " Not Aligned", (w-150, 30), font, 0.9, red, 2)
            
        except:
            pass
         # Render detections
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        cv2.imshow('Mediapipe Feed', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()



libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: fai